# Create pixel-level snow dynamics results from IMS Snow Cubes

Thinking about memory intensive xarray operations (e.g., rolling):
- https://discourse.pangeo.io/t/xarray-unable-to-allocate-memory-how-to-size-up-problem/3233/10
- https://github.com/pydata/xarray/issues/3165
- https://discourse.pangeo.io/t/using-dask-client-and-running-out-of-memory/3500/8

What have I learned:
- Smaller chunks can get things working, just more slowly (e.g., chunk in both space and time dimensions)
- Can split cube into spatial sections, run and put together later (faster, but more coding work?)
- Adding construct to basic rolling seems to help some with speed and memory use

In [1]:
import numpy as np
#import hvplot.xarray
import xarray as xr

import sys
sys.path.append('C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/UtilityCode/DataAccess/Utilities')
import PreProcess_Utils as pputil

sys.path.append('C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/UtilityCode/Modelling/Utilities')
import Snow_Utils as snow_util

In [2]:
from dask.distributed import LocalCluster

cluster = LocalCluster(n_workers = 10, threads_per_worker = 10, memory_limit = 0) # Refined through testing
client = cluster.get_client()
client # Can open in other window by clicking 'Dashboard' link below

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 100,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:59003,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 100
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:59060,Total threads: 10
Dashboard: http://127.0.0.1:59065/status,Memory: 0 B
Nanny: tcp://127.0.0.1:59006,


2025-02-05 09:27:14,701 - distributed.nanny - ERROR - 
Traceback (most recent call last):
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls_2025\Lib\site-packages\distributed\utils.py", line 805, in wrapper
    return await func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls_2025\Lib\site-packages\distributed\nanny.py", line 554, in _on_worker_exit
    await self._unregister()
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls_2025\Lib\site-packages\distributed\nanny.py", line 305, in _unregister
    await wait_for(
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls_2025\Lib\site-packages\distributed\utils.py", line 1910, in wait_for
    return await fut
           ^^^^^^^^^
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls_2025\Lib\site-packages\distributed\core.py", line 1259, in send_recv_from_rpc
    return await send_recv(comm=comm, op=key, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mb

# Load Winter Year Snow Cube

In [3]:
path = 'D:/Canada/IMS/IMS_snowCube_winterYear'
winterYear = '2324'

In [4]:
%%time
snowCube = pputil.uploadNC(path + winterYear + '.nc', form = 'dataarray', chunks = {'time': 100, 'x': 200, 'y': 200}) # time = 13
snowCube

CPU times: total: 78.1 ms
Wall time: 904 ms


<xarray.DataArray (time: 366, y: 4568, x: 5372)> Size: 9GB
dask.array<open_dataset-__xarray_dataarray_variable__, shape=(366, 4568, 5372), dtype=uint8, chunksize=(100, 200, 200), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 43kB -2.351e+06 -2.35e+06 ... 3.019e+06 3.02e+06
  * y            (y) float64 37kB 3.843e+06 3.842e+06 ... -7.229e+05 -7.239e+05
    variable     <U18 72B ...
  * time         (time) datetime64[ns] 3kB 2023-08-21 2023-08-22 ... 2024-08-20
    spatial_ref  int64 8B ...

# Run at once with smaller chunks
- 13, 200, 200

In [ ]:
# %%time
# snowCube = snow_util.cleanSnowCube(snowCube, type = 'binary', temporal = 'continuous', thresh = 7, verbose = True)
# snowCube = snow_util.dailySnowCube2SnowDynamics(snowCube, uncertainty = False, verbose = True)
# snowCube = pputil.loadXR(snowCube)
# snowCube

In [ ]:
# %%time
# snowCube_s = snowCube[:,2000:4000, 2000:4000]
# snowCube_s = snow_util.cleanSnowCube(snowCube_s, type = 'binary', temporal = 'continuous', thresh = 7, verbose = True) 
# snowCube_s = pputil.loadXR(snowCube_s)
# snowCube_s

In [ ]:
#np.unique(snowCube_s, return_counts = True)

In [ ]:
# # Create empty xarray Dataset to fill with selected products
# crs = snowCube_s.rio.crs # Some xarray functions remove crs 

# snowDynamics = xr.Dataset(coords = dict(x = ('x', snowCube_s['x'].values),
#                                         y = ('y', snowCube_s['y'].values)))
# snowDynamics.rio.write_crs(crs, inplace = True) # Reapply crs

# snowDynamics

In [ ]:
# %%time
# test = snowCube_s.groupby('time.month').sum('time')
# test

In [ ]:
# test.dtype

In [ ]:
# %%time
# month_len = snowCube_s.time.dt.days_in_month.groupby('time.month').mean()
# month_len

In [ ]:
# %%time
# test1 = (test / month_len) * 100
# test1

In [ ]:
# test1.dtype

In [ ]:
# %%time
# test2 = test1.round().astype('uint8')
# test2

In [ ]:
# test2.dtype

In [ ]:
# C[0].plot()

In [ ]:
# snowDynamics['pSnow'] = test2
# snowDynamics

# Break into 2000x2000 sections
To fit into memory

In [7]:
%%time

snowCube_ss = []

for x in range(3):
    for y in range(3):
        
        # Break itno sections
        print('---------- Section ' + str(x) + ' ' + str(y) + ' ----------')
        snowCube_s = snowCube[:,x*2000:(x+1)*2000, y*2000:(y+1)*2000] # All 9 sections

        # Generate products and load
        snowCube_s = snow_util.cleanSnowCube(snowCube_s, form = 'binary', temporal = 'continuous', thresh = 7, verbose = True)  
        snowCube_s = snow_util.dailySnowCube2SnowDynamics(snowCube_s, uncertainty = False, verbose = True) # Annual snow dynamics
        #snowCube_s = snow_util.dailySnowCube2SnowDynamics(snowCube_s, products = ['pSnow_month'], uncertainty = False, verbose = True) # Monthly snow %
        snowCube_s = pputil.loadXR(snowCube_s)

        # Save sub-products to rasters 
        # snowCube_s['snow_start'].rio.to_raster('IMS_snowStart_winterYear1819_' + str(x) + str(y) + '.tif')
        # snowCube_s['snow_end'].rio.to_raster('IMS_snowEnd_winterYear1819_' + str(x) + str(y) + '.tif')
        # snowCube_s['snow_length'].rio.to_raster('IMS_snowLength_winterYear1819_' + str(x) + str(y) + '.tif')
        # snowCube_s['snow_periods'].rio.to_raster('IMS_snowPeriods_winterYear1819_' + str(x) + str(y) + '.tif')
        # snowCube_s['snow_status'].rio.to_raster('IMS_snowStatus_winterYear1819_' + str(x) + str(y) + '.tif')
        
        snowCube_ss.append(snowCube_s)

---------- Section 0 0 ----------
Calculated rolling median through time of length 13.
Calculated cumulative sum that resets when non-snow is found.
Identified periods when 7+ consecutive snow observations occured (skipping NaN).
Created flipped snowCube to identify non-snow periods.
Calculated cumulative sum that resets when snow is found.
Identified periods when 7+ consecutive non-snow observations occured (skipping NaN).
Removed (NaN) snow and non-snow periods below 7 consecutive observations.
Filled in NaN transition days with nearest neighbor.
Created empty snowDynamics Dataset to fill.
Calculated cumulative sum of binary snowCube (used in multiple products).
Added snow cover start date (# days from Dec 31) to snowDynamics.
Added snow cover end date (# days from Dec 31) to snowDynamics.
Added snow cover length (# days with snow cover) to snowDynamics.
Added snow period count (# seperated snow periods) to snowDynamics.
Added snow status (0 = Regular fall/melt, 1 = Perennial, 2 = In

# Merge and Save Snow Dynamics

In [8]:
%%time
snowDynamics = xr.combine_by_coords(snowCube_ss)
snowDynamics

CPU times: total: 3.52 s
Wall time: 3.56 s


<xarray.Dataset> Size: 295MB
Dimensions:       (y: 4568, x: 5372)
Coordinates:
  * x             (x) float64 43kB -2.351e+06 -2.35e+06 ... 3.019e+06 3.02e+06
  * y             (y) float64 37kB 3.843e+06 3.842e+06 ... -7.229e+05 -7.239e+05
    spatial_ref   int64 8B 0
    variable      <U18 72B 'IMS_Surface_Values'
Data variables:
    snow_start    (y, x) float32 98MB nan nan nan nan nan ... nan nan nan nan
    snow_end      (y, x) float32 98MB nan nan nan nan nan ... nan nan nan nan
    snow_length   (y, x) uint16 49MB 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    snow_periods  (y, x) uint8 25MB 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    snow_status   (y, x) uint8 25MB 3 3 3 3 3 3 3 3 3 3 ... 3 3 3 3 3 3 3 3 3 3

In [ ]:
# Next time, could clip here? Or stick with 9
    # da = da.rio.clip(roi.geometry.values)
    # da.rio.write_nodata(-999, encoded = True, inplace = True)

In [9]:
%%time
snowDynamics['snow_start'].rio.to_raster('IMS_snowStart_winterYear' + winterYear + '.tif')
snowDynamics['snow_end'].rio.to_raster('IMS_snowEnd_winterYear' + winterYear + '.tif')
snowDynamics['snow_length'].rio.to_raster('IMS_snowLength_winterYear' + winterYear + '.tif')
snowDynamics['snow_periods'].rio.to_raster('IMS_snowPeriods_winterYear' + winterYear + '.tif')
snowDynamics['snow_status'].rio.to_raster('IMS_snowStatus_winterYear' + winterYear + '.tif')
# Save pSnow_month as netCDF? Or set of rasters?
#snowDynamics['pSnow'][3].rio.to_raster('test.tif')
# pputil.downloadNC(snowDynamics, 'C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/Canada/Snow Dynamics/IMS (1 km)/Outputs/pSnow_month', 
#                   'IMS_pSnow_month_winterYear' + winterYear + '.nc')

CPU times: total: 2.91 s
Wall time: 4.04 s


In [ ]:
# x = 3605
# y = 4365

# print('Snow status: ' + str(snowDynamics['snow_status'][x,y].values))
# print('Snow periods: ' + str(snowDynamics['snow_periods'][x,y].values))
# print('Snow start: ' + str(snowDynamics['snow_start'][x,y].values))
# print('Snow end: ' + str(snowDynamics['snow_end'][x,y].values))
# print('Snow length: ' + str(snowDynamics['snow_length'][x,y].values))

# Snow Period ID/Cleaning Validation
- Interpret 100 random points and get snow dynamics
- 20 from each winter year
- See 100pixel_SnowDynamics.xlsx

Find method that best captures our interpretation

## Interpret 100 random points and get snow dynamics

In [5]:
# %%time
# # Loading here will save time when getting plots for 100 points
# snowCube = pputil.loadXR(snowCube)
# snowCube

In [6]:
# x = np.random.randint(0, snowCube.shape[1])
# print(x)
# y = np.random.randint(0, snowCube.shape[2])
# print(y)

# print(snowCube[:, x, y]['x'].values) #.rio.set_spatial_dims('x', 'y').rio.reproject('EPSG:4326')
# print(snowCube[:, x, y]['y'].values)

# snowCube[:, x, y].rename('snowCube').hvplot(height = 600, width = 1000)

In [ ]:
#snowCube[:, x, y]

## MedCumSumMethod

In [ ]:
# snowCube_clean = snow_util.cleanSnowCube(snowCube[:, x -5:x+5, y-5:y+5], thresh = 7).interpolate_na(dim = 'time', method = 'nearest')
# snowCube_clean[:, 5, 5].rename('snowCube_clean').hvplot(height = 600, width = 1000) # Same pixel as above

### Median

In [ ]:
# %%time

# window = 13 # Length of Median rolling window
# thresh = 7 # Number of observations in a row to make snow/non-snow period
# crs = snowCube.rio.crs

# # Window-length rolling median
# snowCube_c = snowCube[:, x-5:x+5, y-5:y+5].rolling(time = window, min_periods = 1, center = True).median() #.astype('uint8') # 3D to make sure works

# # Convert to smallest dtype, this has side-effect of changing any 0.5s to 0 (where min_periods is even and matching). Makes sense to do this when edges are summer. 
# snowCube_c = snowCube_c.astype('uint8') # Do here to avoid influence on unique values

# snowCube_c[:, 5, 5].rename('snowCube_clean').hvplot(height = 600, width = 1000) # Same pixel as above

In [ ]:
#np.unique(snowCube_c, return_counts = True)

### Cumsum

In [ ]:
# %%time

# # Create an xarray where cumsum is calculated but resets each time non-snow (0) is found
# cumsum = snowCube_c.cumsum(dim = 'time')
# cumsum_r = cumsum - cumsum.where(snowCube_c == 0).ffill(dim = 'time').fillna(0)

# # Find groups that meet condition
# snow_grps = xr.full_like(cumsum_r, fill_value = 0) # Something to put results in
# snow_grps = xr.where(cumsum_r >= thresh, 1, snow_grps) # At least this many consecutive observation required
# snow_grps = xr.where((cumsum_r > 0) & (cumsum_r < thresh), np.nan, snow_grps) # Other valid sections over 0 becomes NaNs to be filled
# snow_grps = snow_grps.bfill(dim = 'time') #.fillna(1) # Backfill ones previously set to np.nan

# # Create flipped snowCube (Non-snow = 1, Snow = 0)
# snowCube_cf = xr.where(snowCube_c == 1, 0, snowCube_c)
# snowCube_cf = xr.where(snowCube_c == 0, 1, snowCube_cf)

# # Create an xarray where cumsum is calculated but resets each time snow (0) is found
# cumsum = snowCube_cf.cumsum(dim = 'time')
# cumsum_r = cumsum - cumsum.where(snowCube_cf == 0).ffill(dim = 'time').fillna(0)

# # Find groups that meet condition
# nosnow_grps = xr.full_like(cumsum_r, fill_value = 0) # Something to put results in
# nosnow_grps = xr.where(cumsum_r >= thresh, 1, nosnow_grps) # At least this many consecutive observation required
# nosnow_grps = xr.where((cumsum_r > 0) & (cumsum_r < thresh), np.nan, nosnow_grps) # Other valid sections over 0 becomes NaNs to be filled
# nosnow_grps = nosnow_grps.bfill(dim = 'time') #.fillna(1) # Backfill ones previously set to np.nan

# # Use thresholded groups to clean
# snowCube_c = xr.where(snow_grps == 1, 1, np.nan).astype('float32') # >= thresh cumulative snow observations = 1
# snowCube_c = xr.where(nosnow_grps == 1, 0, snowCube_c) # >= thresh cumulative non-snow observations = 0

# # Only need to do there if there are NaNs in original snow Cube (not the case for IMS)
# #snowCube_c = xr.where(snowCube[:, x-5:x+5, y-5:y+5].isnull(), np.nan, snowCube_c) # Reset np.nans (otherwise will set snow/no-snow in uncertainty period)

# # For IMS (no NaNs only), nearest neighbor interpolate
# snowCube_c = snowCube_c.interpolate_na(dim = 'time', method = 'nearest').astype('uint8') # For larger cube, see chunking setup in dailySnowCube()
# # If 1 and 0 are odd distance apart, then middle date will remain NA and be cast to 0 with astype call - with warning posted. 

# snowCube_c.rio.write_crs(crs, inplace = True) # Reapply crs
 
# snowCube_c[:, 5, 5].rename('snowCube_clean').hvplot(height = 600, width = 1000) # Same pixel as above

## Get Snow Dynmaics

In [ ]:
# %%time

# snowDynamics = snow_util.dailySnowCube2SnowDynamics(snowCube_c, uncertainty = False, verbose = False)
# #snowDynamics

# print('Snow status: ' + str(snowDynamics['snow_status'][5,5].values))
# print('Snow periods: ' + str(snowDynamics['snow_periods'][5,5].values))
# print('Snow start: ' + str(snowDynamics['snow_start'][5,5].values))
# print('Snow end: ' + str(snowDynamics['snow_end'][5,5].values))
# print('Snow length: ' + str(snowDynamics['snow_length'][5,5].values))

## All together (quick check)

In [ ]:
# x = 1025 #np.random.randint(0, snowCube.shape[1])
# y = 2302 # np.random.randint(0, snowCube.shape[2])

# window = 13 # Length of Median rolling window
# thresh = 7 # Number of observations in a row to make snow/non-snow period
# crs = snowCube.rio.crs

# # Window-length rolling median
# snowCube_c = snowCube[:, x-5:x+5, y-5:y+5].rolling(time = window, min_periods = 1, center = True).median() #.astype('uint8') # 3D to make sure works

# # Convert to smallest dtype, this has side-effect of changing any 0.5s to 0 (where min_periods is even and matching). Makes sense to do this when edges are summer. 
# snowCube_c = snowCube_c.astype('uint8') # Do here to avoid influence on unique values

# # Create an xarray where cumsum is calculated but resets each time non-snow (0) is found
# cumsum = snowCube_c.cumsum(dim = 'time')
# cumsum_r = cumsum - cumsum.where(snowCube_c == 0).ffill(dim = 'time').fillna(0)

# # Find groups that meet condition
# snow_grps = xr.full_like(cumsum_r, fill_value = 0) # Something to put results in
# snow_grps = xr.where(cumsum_r >= thresh, 1, snow_grps) # At least this many consecutive observation required
# snow_grps = xr.where((cumsum_r > 0) & (cumsum_r < thresh), np.nan, snow_grps) # Other valid sections over 0 becomes NaNs to be filled
# snow_grps = snow_grps.bfill(dim = 'time') #.fillna(1) # Backfill ones previously set to np.nan

# # Create flipped snowCube (Non-snow = 1, Snow = 0)
# snowCube_cf = xr.where(snowCube_c == 1, 0, snowCube_c)
# snowCube_cf = xr.where(snowCube_c == 0, 1, snowCube_cf)

# # Create an xarray where cumsum is calculated but resets each time snow (0) is found
# cumsum = snowCube_cf.cumsum(dim = 'time')
# cumsum_r = cumsum - cumsum.where(snowCube_cf == 0).ffill(dim = 'time').fillna(0)

# # Find groups that meet condition
# nosnow_grps = xr.full_like(cumsum_r, fill_value = 0) # Something to put results in
# nosnow_grps = xr.where(cumsum_r >= thresh, 1, nosnow_grps) # At least this many consecutive observation required
# nosnow_grps = xr.where((cumsum_r > 0) & (cumsum_r < thresh), np.nan, nosnow_grps) # Other valid sections over 0 becomes NaNs to be filled
# nosnow_grps = nosnow_grps.bfill(dim = 'time') #.fillna(1) # Backfill ones previously set to np.nan

# # Use thresholded groups to clean
# snowCube_c = xr.where(snow_grps == 1, 1, np.nan).astype('float32') # >= thresh cumulative snow observations = 1
# snowCube_c = xr.where(nosnow_grps == 1, 0, snowCube_c) # >= thresh cumulative non-snow observations = 0

# # Only need to do there if there are NaNs in original snow Cube (not the case for IMS)
# #snowCube_c = xr.where(snowCube[:, x-5:x+5, y-5:y+5].isnull(), np.nan, snowCube_c) # Reset np.nans (otherwise will set snow/no-snow in uncertainty period)

# # For IMS (no NaNs only), nearest neighbor interpolate
# snowCube_c = snowCube_c.interpolate_na(dim = 'time', method = 'nearest').astype('uint8') # For larger cube, see chunking setup in dailySnowCube()
# # If 1 and 0 are odd distance apart, then middle date will remain NA and be cast to 0 with astype call - with warning posted. 

# snowCube_c.rio.write_crs(crs, inplace = True) # Reapply crs

# snowDynamics = snow_util.dailySnowCube2SnowDynamics(snowCube_c, uncertainty = False, verbose = False)
# #snowDynamics

# print('Snow status: ' + str(snowDynamics['snow_status'][5,5].values))
# print('Snow periods: ' + str(snowDynamics['snow_periods'][5,5].values))
# print('Snow start: ' + str(snowDynamics['snow_start'][5,5].values))
# print('Snow end: ' + str(snowDynamics['snow_end'][5,5].values))
# print('Snow length: ' + str(snowDynamics['snow_length'][5,5].values))

## Testing Methods
- scipy.signal: argrel (min/max) - only finds 1 length noise, find_peaks (can find center index of peaks - and valleys? but only functions on 1D array (would need to for loop for all pixels)
- Decision tree does not seem viable

In [ ]:
# thresh = 7

In [ ]:
# medCube = snowCube[:, x, y].rolling(time = 15, min_periods = 1, center = True).median()#.dropna('time')#.astype('uint8')
# #cleanCube
# medCube.rename('cleanCube').hvplot(height = 800, width = 1200)

In [ ]:
# # Calculate snow cumsum
# cumsum_s = medCube.cumsum(dim = 'time') 
# cumsum_s.plot()

In [ ]:
# # Calculate resetting cumsum
# cumsum_s_r =  cumsum_s - cumsum_s.where(medCube == 0).ffill(dim = 'time').fillna(0)
# cumsum_s_r.plot()

In [ ]:
# import xarray as xr

In [ ]:
# s_grps = xr.full_like(cumsum_s_r, fill_value = 0) # Something to put results in
# s_grps = xr.where(cumsum_s_r >= thresh, 1, s_grps) # At least this many consecutive observation required
# s_grps = xr.where((cumsum_s_r > 0) & (cumsum_s_r < thresh), np.nan, s_grps) # Other valid sections over 0 becomes NaNs to be filled
# s_grps = s_grps.bfill(dim = 'time') #.fillna(1) # Backfill ones previously set to np.nan
# s_grps.rename('s_grps').hvplot(height = 800, width = 1200)

In [ ]:
# medCube_flip = xr.where(medCube == 1, 0, medCube)
# medCube_flip = xr.where(medCube == 0, 1, medCube_flip)
# medCube_flip.plot()

In [ ]:
# cumsum_ns = medCube_flip.cumsum(dim = 'time')
# cumsum_ns_r = cumsum_ns - cumsum_ns.where(medCube_flip == 0).ffill(dim = 'time').fillna(0)
# cumsum_ns_r.plot()

In [ ]:
# ns_grps = xr.full_like(cumsum_ns_r, fill_value = 0) # Something to put results in
# ns_grps = xr.where(cumsum_ns_r >= thresh, 1, ns_grps) # At least this many consecutive observation required
# ns_grps = xr.where((cumsum_ns_r > 0) & (cumsum_ns_r < thresh), np.nan, ns_grps) # Other valid sections over 0 becomes NaNs to be filled
# ns_grps = ns_grps.bfill(dim = 'time') #.fillna(1) # Backfill ones previously set to np.nan
# ns_grps.rename('ns_grps').hvplot(height = 800, width = 1200)

In [ ]:
# cleanCube = xr.where(s_grps == 1, 1, np.nan)#.astype('float32') # >= thresh cumulative snow observations = 1
# cleanCube = xr.where(ns_grps == 1, 0, cleanCube) # >= thresh cumulative non-snow observations = 0
# cleanCube = xr.where(snowCube[:, x, y].isnull(), np.nan, cleanCube) # Reset np.nans (otherwise will set snow/no-snow in uncertainty period)
# cleanCube.rename('cleanCube').hvplot(height = 800, width = 1200)

In [ ]:
# cleanCube = cleanCube.interpolate_na(dim = 'time', method = 'nearest')
# cleanCube.rename('cleanCube').hvplot(height = 800, width = 1200)

In [ ]:
# from scipy.signal import argrelmin, argrelmax, find_peaks

In [ ]:
# snowCube[:, x-1:x+1, y-1:y+1]

In [ ]:
# argrelmin(snowCube[:, x, y].values, order = 1)

In [ ]:
# %%time
# # #argrelmin(snowCube[:, x, y].values)
# # #test = argrelmin(snowCube[:, x-1:x+1, y-1:y+1].values, order = 4) # Does not find minimums more than one observation wide
# #test = find_peaks(snowCube[:, x, y].values)
# #test = find_peaks(snowCube[:, x-1:x+1, y-1:y+1].values)
# test = find_peaks(snowCube[:, x, y].values, plateau_size = 5) # Flip 0 and 1 to find opposite peaks
# test

In [ ]:
# snowCube[test[0], x, y]

In [ ]:
# dates = snowCube[:, x, y].time.values
# dates = [date[0:10] for date in np.datetime_as_string(dates)]

# obs = snowCube[:, x, y].values
# obs = [ob.item() for ob in obs]

# {dates: obs for dates, obs in zip(dates, obs)}

In [ ]:
# from sklearn import tree

In [ ]:
# X = [[0, 0], [1, 1]]
# Y = [0, 1]
# clf = tree.DecisionTreeClassifier()
# clf = clf.fit(X, Y)
# clf

In [ ]:
# clf.predict(snowCube[:, x-1:x+1, y].values)

In [ ]:
# %%time

# # https://github.com/pydata/xarray/issues/3715

# import numpy as np
# import xarray as xr
# from scipy.signal import find_peaks

# # Generate waveform
# x = (np.sin(2*np.pi*(2**np.linspace(2,10,1000))*np.arange(1000)/48000)
#      + np.random.normal(0, 1, 1000) * 0.15)

# # Find peaks non-xarray way
# peaks, _ = find_peaks(x, prominence=1)
# print(peaks)

# # Cast waveform to xr.DataArray
# x = xr.DataArray(x, dims='time')

# # Duplicate data along a new dimension
# rep = xr.DataArray(range(11), dims='repeat')
# x = (x.broadcast_like(rep).assign_coords(repeat=rep))

# def process_peaks(arr):
#     # Apply find_peaks
#     peaks, _ = find_peaks(arr)
#     return peaks

# # Apply function to array
# results = xr.apply_ufunc(
#     process_peaks, x,
#     input_core_dims = [['time']],
#     output_core_dims = [['peaks']],
#     vectorize = True
# )

# # Should show repeats of peak results
# results


In [ ]:
# import xarray as xr
# from scipy.signal import find_peaks

# def findPeaks(arr):
#     peaks, _ = find_peaks(arr)
#     return peaks

# snow_periods = xr.apply_ufunc(findPeaks, snowCube_clean, input_core_dims = [['time']], output_core_dims = [['peaks']], vectorize = True).count(dim = 'peaks')
# #snow_periods = snow_periods

# snow_periods